### SAP Materials data - Semantic Search

Andre Pretorius 2023-10-15
This file performs semantic search of SAP materials data. To do this, it builds transformer embeddings on the reference dataset, and compares a given search term to these embeddings using a cosine similarity metric.


In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import time
import pickle
from pathlib import Path

In [2]:
# This cell is not used here, it is just an FYI on how to collect the materials data from SAP.
# Raw tables can be exported with SAP transaction SE16 if they are not part of a data pipeline.

materials_tables = {
    'MARA': 'General Material Data This table contains general information about materials, such as material number, material type, industry sector, and creation date.',
    'MAKT': 'Material Descriptions This table stores material descriptions in different languages.',
    'MARC': 'Plant Data for Material This table contains plant-specific data for materials, such as plant, storage location, and procurement type.',
    'MARD': 'Storage Location Data for Material This table stores information about material stock at different storage locations within a plant.',
    'MBEW': 'Material Valuation This table contains valuation data for materials, such as standard price, moving average price, and price unit.',
    'MBEW': 'Material Valuation This table contains valuation data for materials, such as standard price, moving average price, and price unit.',
    'MEAN': 'International Article Number (EAN/UPC) for Material This table stores the International Article Number (EAN) or Universal Product Code (UPC) for materials.',
    'MLGN': 'Material Data for Each Warehouse Number This table contains warehouse-specific material data, such as warehouse number and storage type.',
    'MLGT': 'Material Data for Each Storage Type This table stores material data for each storage type within a warehouse, such as storage bin and maximum storage capacity.',
    'MVKE': 'Sales Data for Material This table contains sales-related data for materials, such as sales organization, distribution channel, and minimum order quantity.',
}

schema_name = 'silver'

query = """
    SELECT
    MARA.MANDT, MARA.MATNR, MARA.ERSDA, MARA.ERNAM, MARA.LAEDA, MARA.AENAM, MARA.MTART,
    MARA.MBRSH, MARA.MATKL, MARA.MEINS, MARA.BSTME, MARA.PTEXT, MAKT.MAKTX
    FROM {schema_name}.MARA
    LEFT JOIN {schema_name}.MAKT ON MARA.MATNR = MAKT.MATNR
    WHERE MAKT.SPRAS = 'E'
    """

In [ ]:
# Consolidated materials data from SAP exported from the datamanagement SQL scripts.
# This data is proprietary so has not been included here.

materials_data_path = Path('proprietary') / 'materials_data.xlsx'
df = pd.read_excel(materials_data_path)
total_number_of_materials = len(df)
total_number_of_materials

In [7]:
# Cleanup the data, or get a smaller subset for testing

def sanitise_data(df, target_items=None):
    df = df.copy()
    df['encoding_text'] = (
        df['PTEXT'].astype(str).str.upper() + '; ' + df['MAKTX'].astype(str).str.upper()
    )
    df = df[~df['encoding_text'].str.contains('BLOCKED')]

    if target_items:
        current_len = len(df)
        slice_interval = max(current_len // target_items, 1)
        df = df.iloc[::slice_interval].reset_index()

    return df

In [8]:
# Functions to get the embeddings for semantic search
embeddings_file = Path('proprietary') / 'embeddings.pickle'

model = SentenceTransformer('paraphrase-distilroberta-base-v1')


def get_duration_string(duration):
    hours, remainder = divmod(duration, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds"


def encode_and_save_embeddings(df, encoding_column, embeddings_file):
    start_time = time.time()

    embeddings = model.encode(df[encoding_column])

    with open(embeddings_file, 'wb') as f:
        pickle.dump(embeddings, f)

    end_time = time.time()
    duration = end_time - start_time
    num_embeddings = len(df)

    predicted_duration = duration * total_number_of_materials / num_embeddings
    print(
        f"Embeddings completed for {num_embeddings} entries in {get_duration_string(duration)}"
    )
    print(
        f"Predicted time for all {total_number_of_materials} entries is {get_duration_string(predicted_duration)}"
    )

    return embeddings


def load_embeddings(embeddings_file):
    with open(embeddings_file, 'rb') as f:
        embeddings = pickle.load(f)
    return embeddings

In [ ]:
# Perform the embeddings. Pass a value to target_items to just encode a subset for testing purposes.
df_short = sanitise_data(df, target_items=None)

res = input('Type X to reload embeddings and save to disk.')
if res.upper() == 'X':
    embeddings = encode_and_save_embeddings(
        df_short, encoding_column='encoding_text', embeddings_file=embeddings_file
    )
else:
    embeddings = load_embeddings(embeddings_file)

In [41]:
def semantic_search(df, query, embeddings, top_n=10):
    print(len(embeddings))
    query = query.upper()
    query_embedding = model.encode([query])
    similarity_scores = cosine_similarity(query_embedding, embeddings)
    sorted_indices = similarity_scores.argsort()[0][::-1][:top_n]
    top_scores = similarity_scores[0][sorted_indices]

    results = df.loc[sorted_indices].copy()
    results['similarity_score'] = top_scores
    return results[['MATNR', 'PTEXT', 'MAKTX', 'similarity_score']]


def literal_search(df, query):
    df_results = df.copy()
    query_terms = [item.strip(' .,').upper() for item in query.split(' ')]
    df_results['search_terms'] = df_results['encoding_text'].astype(str)
    df_results['search_mask'] = True
    for query_term in query_terms:
        df_results['search_mask'] = df_results['search_mask'] & df_results[
            'encoding_text'
        ].str.contains(query_term)

    return df_results[df_results['search_mask']][['MATNR', 'PTEXT', 'MAKTX']]

#### Semantic search


In [ ]:
query = "gasket spiral"
results = semantic_search(df_short, query, embeddings)
print("Top matches using semantic search:")
results

(ouput removed)


#### Literal search


In [ ]:
results = literal_search(df_short, query)
print("Top matches using literal search:")
results

(ouput removed)
